In [ ]:
import ray
ray.init()
# ray.shutdown()

In [58]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)

import pandas_bokeh
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import polygon_ds as pds
import polygon_df as pdf
import polygon_s3 as ps3
import ray_symbol_details as rsd
import market_preprocess as mp
import market_cov as mcov
import market_cluster as mc
import utils_pickle as up
# import ipdb
# ipdb.set_trace(context=10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
start_date = '2018-10-24'
end_date = '2020-11-24'
symbol = 'market'
tick_type='daily'

In [62]:
r = mp.prepare_data(start_date, end_date, beta_symbol='VTI')

4502494 Initial rows 11509 symbols
-1460110 all dates filter 5784 symbols
-383980 liquidity filter 5054 symbols
-125188 volitility filter 4816 symbols
-24196 min $value filter 4770 symbols
-706944 symbol details filter 3426 symbols
1802076 Final rows 40.0 % remaining


100%|██████████| 3426/3426 [00:24<00:00, 138.00it/s]


In [63]:
r.keys()

dict_keys(['df', 'sym_meta', 'close', 'returns', 'log_returns', 'zs_log_returns', 'g_zs_log_returns', 'g_zs_log_returns_resid'])

In [69]:
dist_mat = mcov.cov_denoise_detone_dist(r['g_zs_log_returns_resid'], detone=False)

In [24]:
kmean_labels = mc.get_kmean_clusters(dist_mat, n_clusters=11)

mc.evaluate_clustering(labels_true=r['sym_meta'].sector.values, labels_pred=kmean_labels)

{'ari': 0.1504595975132633, 'ami': 0.25454594601518826}

In [25]:
kmean_labels = mc.get_hira_clusters(dist_mat, n_clusters=11)

mc.evaluate_clustering(labels_true=r['sym_meta'].sector.values, labels_pred=kmean_labels)

{'ari': 0.1566378887498753, 'ami': 0.2724050154916307}

In [8]:
kmean_labels = mc.get_hira_clusters(r['g_zs_log_returns_resid'].transpose(), n_clusters=11)

mc.evaluate_clustering(labels_true=r['sym_meta'].sector.values, labels_pred=kmean_labels)

{'ari': 0.15414956514580322, 'ami': 0.3232448242377991}

In [13]:
kmean_labels = mc.get_kmean_clusters(r['g_zs_log_returns_resid'].transpose(), n_clusters=11)

mc.evaluate_clustering(labels_true=r['sym_meta'].sector.values, labels_pred=kmean_labels)

{'ari': 0.19124218050351974, 'ami': 0.3409286306986541}

In [ ]:
kmean_labels = mc.get_spectral_clusters(r['g_zs_log_returns_resid'].transpose(), n_clusters=11)

mc.evaluate_clustering(labels_true=r['sym_meta'].sector.values, labels_pred=kmean_labels)

In [ ]:
kmean_labels = mc.get_optics_clusters(dist_mat)

mc.evaluate_clustering(labels_true=r['sym_meta'].sector.values, labels_pred=kmean_labels)

In [ ]:
corex_labels = mc.get_corex_clusters(r['g_zs_log_returns_resid'], n_clusters=11)

mc.evaluate_clustering(labels_true=r['sym_meta'].sector.values, labels_pred=corex_labels)

In [ ]:
corex_labels = mc.get_corex_clusters(dist_mat, n_clusters=11)

mc.evaluate_clustering(labels_true=r['sym_meta'].sector.values, labels_pred=corex_labels)

In [ ]:
corex300 = up.pickle_load('data/corex300.pkl')

# X = df.reset_index(drop=True)
# corex, full_df = mc.corex_fit(X, n_hidden=200)

# up.pickle_dump((corex, full_df), file_name='corex200.pkl')

# gsutil copy /Users/bobcolner/QuantClarity/pandas-polygon/corex200.pkl gs://emerald-skill-datascience/

In [ ]:
import h2o
from h2o.estimators import H2OGeneralizedLowRankEstimator
h2o.init()

# Import the USArrests dataset into H2O:
arrestsH2O = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/pca_test/USArrests.csv")

# Split the dataset into a train and valid set:
train, valid = arrestsH2O.split_frame(ratios=[.8], seed=1234)

# Build and train the model:
glrm_model = H2OGeneralizedLowRankEstimator(
    k=4,
    loss="Huber",
    loss_by_col={'cat_var': 'Categorical'},
    regularization_x="OneSparse", # None (default), Quadratic, L2, L1, NonNegative, OneSparse, UnitOneSparse, Simplex.
    regularization_y="OneSparse",
    gamma_x=0.5,
    gamma_y=0.5,
    max_iterations=1000,
    recover_svd=True,
    init="PlusPlus",
    transform="standardize",
   )
glrm_model.train(training_frame=train)

In [ ]:
### market clustering

# %time df = read_market_daily(result_path)
mdf = pd.read_parquet('data/old/mdf.parquet')

%time npdf, sharpe_ratios = mc.normalize_market_df(mdf)

# %time npdf_resid = colwise_linreg(npdf)
npdf_resid = pd.read_parquet('data/old/npdf_resid.parquet')

# %time par_cor_mat = npdf_resid.corr()
resid_par_cor_mat = pd.read_parquet('data/old/partial_cor_mat.parquet')

details_df = pd.read_parquet('data/old/details_df.parquet')

# cluster_lables = cluster_sim_matrix(similarity=resid_par_cor_mat.abs())
cluster_lables = pd.read_parquet('data/old/cluster_lables.parquet')

# symbol_meta = join_symbol_data(details_df, cluster_lables, sharpe_ratios, mdf)
symbol_meta = pd.read_parquet('data/old/symbol_meta.parquet')

# cluster_coheasion = mc.get_cluster_coheasion(sim_df=resid_par_cor_mat.abs(), symbol_meta=symbol_meta, cluster_col='cluster_n200')

In [ ]:
symbol_meta['liquidy_rank'] = symbol_meta.groupby('cluster_n200')['daily_avg_dollar_volume'].rank(method='first')

symbol_meta = symbol_meta.sort_values(['cluster_n200', 'liquidy_rank']).reset_index()

by_cluster = symbol_meta.groupby('cluster_n200')
result = []
for cluster, frame in by_cluster:
    counts = dict(frame['sector'].value_counts())
    counts.pop('', None)  # drop symbols w/o sector/industry info
    result.append(counts)

In [ ]:
cluster_sec = pd.DataFrame(result)    
cluster_sec = cluster_sec.drop(columns=['Finance'])
cluster_sec_pct = cluster_sec.apply(lambda x: 100 * x / float(x.sum()), axis=1)

cluster_sec_pct.style.background_gradient(cmap='coolwarm', axis=0)

# n += 1
n = 61
print(n)
symbol_meta[symbol_meta.cluster_n200 == n].sector.value_counts()

sym_clust = symbol_meta[symbol_meta.cluster_n200 == n]
sym_clust